## Variaciones sobre el modelo inicial 

Vamos a implementar alguna modificación sobre el modelo inicial propuesto en la primera sesión.

Para empezar vamos a definir un conjunto de validación. Con este conjunto de calidación extraído del propio conjunto de entrenamiento estimaremos los mejores parámetros. Dejaremos el conjunto de test sólo para estimar el acierto final con estos mejores parámetros.

Primero leemos los datos y normalizamos:

In [ ]:
from tensorflow import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Partición training/validación

Nos quedaremos con un 80% de los datos para entrenar y un 20% para validar:

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print('training set', x_train.shape)
print('val set', x_val.shape)

## Experimentación con conjunto de validación

Cualquier parámetro a modificar en nuestro modelo debería ser probado sobre el conjunto de validación y escoger aquel que produjera el mejor resultado para probar ese (y sólo ese) sobre el conjunto de test.

In [ ]:
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD

batch_size=128
epochs=5  

hdim=[128,256,512,1024]
best_acc=0.0
for h in hdim:
    model = Sequential()

    model.add(Input(784))
    model.add(Dense(h, activation='relu')) # <-- repetir esta línea tantas veces como número de capas ocultas se quiera
    model.add(Dense(num_classes, activation='softmax'))

    sgd=SGD(learning_rate=0.01, momentum=0.9)


    model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
    
 

    history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val)) ## <--- OJO validation set
    
    if history.history['val_accuracy'][-1]>best_acc:
        best_acc=history.history['val_accuracy'][-1]
        besth=h

print("=============================")
print("Best acc",best_acc)
print("Best hidden dim",besth)
print("=============================")


## PROBAR EL MEJOR CON TEST
model = Sequential()

model.add(Input(784))
model.add(Dense(besth, activation='relu'))  
model.add(Dense(num_classes, activation='softmax'))

sgd=SGD(learning_rate=0.01, momentum=0.9)


model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

history = model.fit(x_train, y_train, # <--- aquí podríamos concatenar trainin+valid para no malgastar datos
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))  




## EARLY STOPPING

De la multitud de parámetros a decidir en la definición y entrenamiento de las redes neuronales hay uno que podríamos automatizar. Es el número de epochs a emplear. Dado que tenemos un conjunto de validación vamos a emplearlo para monitorizar cómo evoluciona la convergencia. Si en algún momento el descenso por gradiente parece haber alcanzado una zona de meseta sobre dicho conjunto de validación podremos detener el entrenamiento.

In [ ]:
model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

sgd=SGD(learning_rate=0.01, momentum=0.9)

callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3) ## También podría ser 'loss' sin necesitar un conjunto de validación

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

epochs=100  ## No nos preocupemos, el fit acabará antes por el early_stopping
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[callback])  



## **Ejercicio**:

Entrenar una red neuronal MLP para el problema MNIST. Los valores que se quieren probar son los siguientes:

-   Número de capas ocultas [1,2,3]
-   Dimensión de las capas ocultas [512,1024]
-   Learning rate [0.01, 0.001, 0.0001]
-   Batch_size [128]
-   Epochs con early_stopping

Obtener la mejor combinación con el conjunto de validación y probarla finalmente sobre el conjunto de test